In [1]:
from __future__ import division
# import random
# #num_samples = 100000000
# num_samples = 10000
# def inside(p):     
#   x, y = random.random(), random.random()
#   return x*x + y*y < 1
# count = sc.parallelize(range(0, num_samples)).filter(inside).count()
# pi = 4 * count / num_samples
# print(pi)
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql import Row

In [2]:
def wordCount(sparkSession, input_folder):

#     lines = sparkSession.readStream.option('inferSchema', True).csv(input_folder)
    lines = sparkSession.readStream.text(input_folder)
#     parts = lines.map(lambda l: l.split(","))
#     DATE,TIME,PICK UP ADDRESS
#     uber = parts.map(lambda p: Row(date=p[0], time=int(p[1]), pick_up_address=p[2]))
#     words = lines.asString.flatMap()
#     Split the lines into words

    words = lines.select(explode(split(lines.value, " ")).alias("word"))
    sql = 'SELECT value, COUNT(*) as count FROM words GROUP BY value'
    wordCounts = words.groupBy("word").count()
    query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option('truncate', False) \
    .option('numRows', 100) \
    .start()

    query.awaitTermination()

In [ ]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

wordCount(spark, 'file:///Users/phxlk/Workspace/spark-meetup/data')
spark.stop()